### <center>Zadanie 4</center>
#### Grupowanie na zbiorze dotyczącym marskości wątroby

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
import seaborn as sns

### Wczytanie danych

In [ ]:
data = pd.read_csv('cirrhosis.csv')
data.drop(columns=['ID', 'Status'], inplace=True)
data['Stage'] = data['Stage'].astype('category')

data.info()

In [ ]:
data.head()

### Wizualizacja danych

<ol>
<li>wykresy kołowe</li>
<li>histogramy</li>
<li>macierz korelacji</li>
</ol>

### Przygotowanie danych

<ol>
<li>konwersja object do category</li>
<li>uzupełnienie danych</li>
<li>standard scaler na kolumnach float, int</li>
</ol>

#### Wykresy kołowe przedstawiają podział pacjentów według:
<ol>
<li>przyjętego lekarstwa</li>
<li>płci</li>
<li>obecności wodobrzusza</li>
<li>obecności hepatomegalii</li>
<li>obecności pajączków naczyniowych</li>
<li>obecność obrzęku w połączeniu z leczeniem diuretykami</li>
</ol>

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(12, 16))
plt.suptitle('Podział pacjentów według kategorii', fontsize=20)

colors = ['blue', 'yellow', 'gray', 'red', 'black']
titles = ['Przyjmowanie penicylaminy', 'Płeć', 'Wodobrzusze', 'Hepatomegalia', 'Pajączki naczyniowe', 'Obrzęk z leczeniem diuretykami', 'Stopień marskości wątroby']
object_columns = data.select_dtypes(include=['object']).columns.values
object_columns = np.append(object_columns, 'Stage')

for i, column in enumerate(object_columns):
    x, y = divmod(i, 2)
    el = data.groupby(column, dropna=False, as_index=True, observed=True).size()
    explode = [0.05 for _ in range(len(el))]
    ax[x, y].pie(
        x=el,
        explode=explode,
        labels=el.index,
        colors=colors,
        autopct='%1.1f%%',
        textprops={'fontsize': 12, 'fontweight': 'bold'},
        radius=1,
        startangle=180,
        labeldistance=1.2,
        wedgeprops={'edgecolor': 'black', 'linewidth': 2},
        normalize=True,
    )
    ax[x, y].set_title(f'{titles[i]}', fontsize=14)

ax[3, 1].axis('off')
plt.tight_layout()
plt.show()

#### Konwersja object na category + Uzupełnienie danych kategorycznych

Uzupełnienie wartości następuje losowo zachowując procenty udziału tych kategorii w liczebności bez NaN.

In [ ]:
encoders = [LabelEncoder() for _ in range(len(object_columns))]

for i, column in enumerate(object_columns):
    counts = data[column].value_counts()
    missing_count = data[column].isna().sum()
    counts *= (missing_count / counts.sum())
    counts = np.ceil(counts).astype(int)
    series = pd.Series(np.repeat(counts.index, counts))[:missing_count]
    series = series.sample(frac=1, random_state=42).reset_index(drop=True)
    data.loc[data[column].isna(), column] = series.values
    data[column] = encoders[i].fit_transform(data[column])
    data[column] = data[column].astype('category')

data.info()

#### Histogramy przedstawiają rozkład danych w kolumnach:

<ol>
<li>N_Days -> liczby dni między rejestracją, a wcześniejszym zgonem, przeszczepem lub końcem analizy badania</li>
<li>Age -> wiek pacjenta w dniach</li>
<li>Bilirubin -> stężenie bilirubiny (mg/dl)</li>
<li>Choresterol -> stężenie choresterolu (mg/dl)</li>
<li>Albumin -> albumina (mg/dl)</li>
<li>Copper -> ilość miedzi w moczu (µg/dzień)</li>
<li>Alk_Phos -> fosfataza alkaliczna (U/litr)</li>
<li>SGOT -> aminotransferaza asparaginianowa (U/ml)</li>
<li>Tryglicerides -> liczba triglicerydów</li>
<li>Platelets -> liczba płytek krwi na ml/1000</li>
<li>Prothrombin -> czas protrombinowy -> czas krzepnięcia krwi</li>
</ol>

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=3, figsize=(16, 12))

titles = ['Liczba dni między rejestracją, a końcem', 'Wiek pacjenta (dzień)', 'Stężenie bilirubiny (mg/dl)', 'Stężenie choresterolu (mg/dl)', 'Obecność albuminy (mg/dl)', 'Ilość miedzy w moczu  (µg/dzień)', 'Fosfataza alkaliczna (U/litr)', 'SGOT (U/ml)', 'Liczba triglicedydów', 'Liczba płytek krwi na ml/1000', 'Czas protrombinowy']

plt.suptitle('Rozkład wartości w kolumnach z zmiennymi liczbowymi', fontsize=20)

for i, column in enumerate(data.select_dtypes(include=np.number).columns[:-1]):
    x, y = divmod(i, 3)
    ax[x, y].hist(
        x=data[column], color='blue',
        edgecolor='black',
        linewidth=1,
        alpha=1
    )
    ax[x, y].set_title(f'Kolumna: {column}', fontsize=14)
    ax[x, y].set_ylabel('Liczebność')

ax[3, 1].axis('off')
ax[3, 2].axis('off')
plt.tight_layout()
plt.show()

#### Uzupełnienie danych numerycznych

Wykorzystanie mediany dla małych braków i interpolacji w reszcie przypadków.
Zastosowanie StandardScalera dla zminiejszenia odległości między punktami w zbiorze (ułatwi to robotę dla kMeans).

In [ ]:
number_columns = data.select_dtypes(exclude='category').columns

for i, column in enumerate(number_columns):
    if data[column].isna().sum() / len(data.index) > 0.05:
        data[column] = data[column].interpolate(method='linear', limit_direction='both')
    else:
        data[column] = data[column].fillna(data[column].median())

standardizer = StandardScaler()
after_standard = standardizer.fit_transform(data[number_columns])
data.drop(columns=number_columns, inplace=True)
data = pd.concat([data, pd.DataFrame(after_standard, columns=number_columns)], axis=1)

data.info()

In [ ]:
data.head()

#### Macierz korelacji Pearsona

In [ ]:
object_columns = data.select_dtypes(include='object').columns.values
object_columns = np.append(object_columns, 'Stage')
encoders = [LabelEncoder() for _ in range(len(object_columns))]

for i, column in enumerate(object_columns):
    data[column] = encoders[i].fit_transform(data[column])

plt.figure(figsize=(12, 10))
sns.heatmap(data.corr(), annot=True, fmt='.2f', cmap='Greys')
plt.show()

### Analiza skupień przy wykorzystaniu algorytmu KMeans

In [ ]:
from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters=3)
kmeans.fit(data)